In [1]:
import pandas as pd
import os
from typing import Union
from functools import reduce


In [2]:
data_path = '..\\data'
file_paths = {
    'jpmx': os.path.join(data_path, 'JPMX_hardcopy.xlsx'),
    'country_risk': os.path.join(data_path, 'Country_Risk_Indices_hardcopy.xlsx'),
    'bbg': os.path.join(data_path, 'Daten_BBG_hardcopy.xlsx'),
    'resource': os.path.join(data_path, 'Rohstoffdaten_hardcopy.xlsx')
}

In [3]:
def process_multi_header_file(
    file_path: str, 
    header_rows: int = 1, 
    rows_to_delete: int = 0,
    sheet_name: Union[str, int, None] = 0, 
    step: int = 1, 
    date_col_index: int = 0,
    data_start_col: int = 1,
    interleaved_dates: bool = False,
    date_format: str | None = None
) -> pd.DataFrame:
    """
    Loads and cleans an Excel file with variable headers and optional row deletion.
    Handles both normal (date + multiple data columns) and interleaved (date, data, date, data...) formats.
    """
    # --- Parameter validation ---
    if header_rows <= 0:
        raise ValueError("header_rows must be a positive integer.")
    if interleaved_dates and step != 2:
        raise ValueError("When interleaved_dates=True, step must be 2.")

    # --- Helper function for cleaning column names ---
    def _clean_column_name(column_header):
        """Convert column header(s) into a clean snake_case-like string."""
        if header_rows == 1:
            return str(column_header).strip().replace(' ', '_')
        parts = [str(part).strip() for part in column_header 
                 if 'Unnamed' not in str(part) and str(part).strip()]
        return '_'.join(parts).replace(' ', '_')

    # --- Read Excel ---
    header_arg = list(range(header_rows)) if header_rows > 1 else 0
    df_raw = pd.read_excel(file_path, header=header_arg, sheet_name=sheet_name)

    if rows_to_delete > 0:
        df_raw = df_raw.iloc[rows_to_delete:]
    
    df_raw = df_raw.dropna(axis=0, how='all').reset_index(drop=True)

    # --- Case 1: Handle interleaved date/data columns ---
    if interleaved_dates:
        processed_dfs = []

        for i in range(0, df_raw.shape[1], 2):
            if i + 1 >= df_raw.shape[1]:
                continue

            col_name = _clean_column_name(df_raw.columns[i + 1])
            if not col_name:
                continue

            temp_df = pd.DataFrame({
                'Date': pd.to_datetime(df_raw.iloc[:, i], errors='coerce', format=date_format),
                col_name: df_raw.iloc[:, i + 1]
            }).dropna(subset=['Date'])
            
            # Filter out implausible dates like 1970 that arise from '0' values
            temp_df = temp_df[temp_df['Date'].dt.year > 1980]

            if not temp_df.empty:
                processed_dfs.append(temp_df)

        if not processed_dfs:
            return pd.DataFrame()

        final_df = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), processed_dfs)
        
        return final_df.sort_values(by='Date').reset_index(drop=True)

    # --- Case 2: Normal date + multiple data columns ---
    else:
        clean_data = {
            'Date': pd.to_datetime(df_raw.iloc[:, date_col_index], errors='coerce', format=date_format)
        }
        for i in range(data_start_col, df_raw.shape[1], step):
            col_name = _clean_column_name(df_raw.columns[i])
            if col_name:
                clean_data[col_name] = df_raw.iloc[:, i]
        
        df_clean = pd.DataFrame(clean_data)
        return df_clean.dropna(subset=['Date']).reset_index(drop=True)

In [4]:
df_jpmx = process_multi_header_file(
    file_paths['jpmx'],
    header_rows=2,
    rows_to_delete=0,
    step=2,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=True
)
df_jpmx.tail(10)

,Date,EMBIG_Div_JPGCSOSD_Index.1,High_Grade_JPGCIGSS_Index.1,High_Yield_JPGCHYSS_Index.1,Africa_JPSSGDAF_Index.1,Asia_JPSSGDAS_Index.1,Europe_JPSSGDER_Index.1,Latin_America_JPSSGDLA_Index.1,Middle_East_JPSSGDME_Index.1
5129,2025-07-17,310.235,106.895,538.121,453.0,105.0,262.0,395.0,256.0
5130,2025-07-18,309.370,107.068,535.836,450.0,105.0,262.0,392.0,257.0
5131,2025-07-21,307.224,106.042,532.839,445.0,104.0,262.0,389.0,256.0
5132,2025-07-22,309.783,109.107,534.801,445.0,106.0,267.0,392.0,258.0
5133,2025-07-23,305.464,106.598,528.120,439.0,103.0,262.0,388.0,254.0
5134,2025-07-24,302.167,105.896,521.427,430.0,101.0,262.0,383.0,254.0
5135,2025-07-25,302.322,106.920,520.577,429.0,101.0,261.0,384.0,254.0
5136,2025-07-28,296.832,103.293,512.556,418.0,97.0,253.0,380.0,250.0
5137,2025-07-29,302.588,108.334,519.836,428.0,103.0,261.0,383.0,256.0
5138,2025-07-30,298.944,105.066,515.766,423.0,98.0,256.0,381.0,251.0


In [5]:
df_resource_energy = process_multi_header_file(
    file_paths['resource'], 
    header_rows=2,
    rows_to_delete=0,
    sheet_name="Energierohstoffe",
    step=2,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=True
)
    
df_resource_metals = process_multi_header_file(
    file_paths['resource'], 
    header_rows=2,
    rows_to_delete=0,
    sheet_name="Metalle", 
    step=2,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=True
)

df_resource_agrar = process_multi_header_file(
    file_paths['resource'], 
    header_rows=2,
    rows_to_delete=0,
    sheet_name="Agrarprodukte", 
    step=2,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=True
)
df_resource_metals

,Date,Gold_GC1_COMB_Comdty.1,Silver_SI1_COMB_Comdty.1,Platinum_PL1_COMB_Comdty.1,Aluminum_LA1_Comdty.1,Kupfer_HG1_COMB_Comdty.1,Blei_LL1_Comdty.1,Nickel_LN1_Comdty.1,Zink_LX1_Comdty.1,Cobalt_LMCODY_LME_Comdty.1,Eisenerz_IOE1_COMB_Comdty.1,Uran_UXA1_Comdty.1,Bitcoin_XBTUSD_BGN_Curncy.1,Zinn_LT1_Comdty.1
0,2005-01-03,429.7,6.507,857.5,NaN,147.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-01-04,429.2,6.452,847.8,1807.00,134.30,943.00,14210.00,1168.00,NaN,NaN,NaN,NaN,7555.00
2,2005-01-05,427.3,6.535,854.3,1829.50,137.50,941.00,14435.00,1206.00,NaN,NaN,NaN,NaN,7205.00
3,2005-01-06,421.6,6.455,847.6,1811.50,137.50,941.00,14555.00,1207.00,NaN,NaN,NaN,NaN,7477.00
4,2005-01-07,419.5,6.450,843.2,1819.50,137.00,955.00,15095.00,1230.00,NaN,NaN,NaN,NaN,7731.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6216,2025-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118004.91,NaN
6217,2025-07-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118801.81,NaN
6218,2025-07-28,3310.0,38.026,1404.1,2632.51,559.50,1993.78,15115.97,2821.80,32889.5,791.5,72.35,118040.22,33703.00
6219,2025-07-29,3324.0,38.084,1404.0,2604.94,560.35,1993.20,15161.38,2807.08,32889.5,791.5,71.15,117479.31,33713.50


In [ ]:
# --- Read country risk indices
df_raw = pd.read_excel(file_paths["country_risk"], header=[0, 1])

# --- Perform cleaning steps
df_raw = df_raw.dropna(axis=0, how='all').reset_index(drop=True)
df_raw = df_raw.drop(df_raw.columns[0], axis=1)

# --- Build the new dataframe from the raw data
cleaned_data = {}
cleaned_data['Date'] = pd.to_datetime(df_raw.iloc[:, 0], format='%d.%m.%Y', errors='coerce')

for i in range(1, df_raw.shape[1], 5):
    country_name = df_raw.columns[i][0].strip()

    for j in range(4):
        risk_col_index = i + 1 + j
        
        risk_type = df_raw.columns[risk_col_index][0].strip()
        index_name = df_raw.columns[risk_col_index][1].strip()

        new_col_name = f"{country_name}_{risk_type}_{index_name}".replace(' ', '_')
        
        # Shift the data by 1 quarter to account for reporting lag
        cleaned_data[new_col_name] = df_raw.iloc[:, risk_col_index].shift(1)

df_country_risk = pd.DataFrame(cleaned_data)

df_country_risk

,Date,Argentina_Country_Risk_RSSCAROR_Index,Argentina_Financial_Risk_RSSCARFR_Index,Argentina_Economic_Risk_RSSCARER_Index,Argentina_Political_Risk_RSSCARPR_Index,Australia_Country_Risk_RSSCAUOR_Index,Australia_Financial_Risk_RSSCAUFR_Index,Australia_Economic_Risk_RSSCAUER_Index,Australia_Political_Risk_RSSCAUPR_Index,Austria_Country_Risk_RSSCASOR_Index,...,Uruguay_Economic_Risk_RSSCUGER_Index,Uruguay_Political_Risk_RSSCUGPR_Index,Venezuela_Country_Risk_RSSCVNOR_Index,Venezuela_Financial_Risk_RSSCVNFR_Index,Venezuela_Economic_Risk_RSSCVNER_Index,Venezuela_Political_Risk_RSSCVNPR_Index,Vietnam_Country_Risk_RSSCVTOR_Index,Vietnam_Financial_Risk_RSSCVTFR_Index,Vietnam_Economic_Risk_RSSCVTER_Index,Vietnam_Political_Risk_RSSCVTPR_Index
0,2009-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-06-30,38.97,69.46,84.28,9.71,93.24,72.14,85.46,93.75,77.82,...,18.42,43.47,7.10,13.59,46.18,4.89,30.44,79.53,52.51,14.26
2,2009-09-30,51.62,67.30,82.44,9.71,86.73,52.71,88.45,93.75,85.93,...,16.52,43.47,10.28,8.43,43.03,4.89,44.98,76.85,50.55,14.26
3,2009-12-31,45.97,55.85,82.19,9.71,88.13,72.18,82.08,93.75,89.88,...,15.07,43.47,6.72,7.96,21.90,4.89,43.11,70.19,53.92,14.26
4,2010-03-31,44.90,58.03,77.43,9.71,87.08,71.04,78.30,93.75,83.78,...,17.74,43.47,12.88,16.14,40.25,4.89,21.78,26.82,38.19,14.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2024-06-30,30.26,79.16,21.50,8.73,89.81,73.00,87.97,92.99,82.57,...,NaN,NaN,1.03,5.37,0.66,3.43,40.17,31.63,73.95,19.95
62,2024-09-30,30.21,75.00,26.70,8.73,90.05,71.86,88.99,92.99,83.56,...,NaN,NaN,0.78,4.51,0.41,3.43,32.16,17.17,69.90,19.95
63,2024-12-31,19.65,57.49,17.55,8.73,93.20,84.41,89.32,92.99,71.40,...,NaN,NaN,0.48,4.84,0.19,3.43,36.87,26.85,69.51,19.95
64,2025-03-31,46.39,89.06,49.80,8.73,87.69,80.69,73.80,92.99,81.96,...,NaN,NaN,0.39,3.49,0.14,3.43,31.32,17.31,65.77,19.95


In [7]:
df_bbg_zinsen = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=2,
    rows_to_delete=2,
    sheet_name="Zinsen", 
    step=1,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=False
)
df_bbg_zinsen

,Date,Zinsen_10Y_DE_GDBR10_Index,Zinsen_10Y_US_USGG10YR_Index,Zinskurve_2Y10Y_DE_DEYC2Y10_Index,Zinskurve_3M10Y_US_USYC3M10_Index,Zinskurve_2Y10Y_DE_USYC2Y10_Index
0,2005-01-03,3.639,4.2104,116.9,196.659,112.238
1,2005-01-04,3.653,4.2886,116.2,200.948,109.297
2,2005-01-05,3.654,4.2808,117,202.766,108.506
3,2005-01-06,3.625,4.2612,118.4,201.646,110.717
4,2005-01-07,3.61,4.269,117.1,198.958,108.244
...,...,...,...,...,...,...
5364,2025-07-25,2.718,4.3878,76.486,3.15,46.036
5365,2025-07-28,2.689,4.4098,76.787,6.692,47.785
5366,2025-07-29,2.708,4.3204,76.355,-2.821,44.945
5367,2025-07-30,2.706,4.37,74.86,2.194,42.528


In [8]:
df_bbg_spreads = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=1,
    rows_to_delete=9,
    sheet_name="Spreads", 
    step=1,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=False
)
df_bbg_spreads

,Date,Spread_Investmentgrade,Spread_High_Yield
0,2005-01-14,0.610737,3.095897
1,2005-01-17,0.610481,3.096612
2,2005-01-18,0.610741,3.135817
3,2005-01-19,0.612084,3.140146
4,2005-01-20,0.616048,3.173013
...,...,...,...
5336,2025-07-24,0.808880,3.076026
5337,2025-07-25,0.801382,3.077771
5338,2025-07-28,0.796797,3.054941
5339,2025-07-29,0.800752,3.090886


In [9]:
df_bbg_inflation = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=1,
    rows_to_delete=0,
    sheet_name="Inflation", 
    step=2,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=True
)
df_bbg_inflation

,Date,US_5Y5Y_Inflation,DE_5Y5Y_Inflation
0,2005-01-03,2.8900,2.3375
1,2005-01-04,2.8800,2.3475
2,2005-01-05,2.8050,2.3800
3,2005-01-06,2.8250,2.3950
4,2005-01-07,2.8450,2.3400
...,...,...,...
5359,2025-07-25,2.5308,2.1404
5360,2025-07-28,2.5310,2.1263
5361,2025-07-29,2.5069,2.1251
5362,2025-07-30,2.5083,2.1242


In [10]:
df_bbg_btc = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=2,
    rows_to_delete=2,
    sheet_name="VIX_Bitcoin", 
    step=1,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=False
)
df_bbg_btc

,Date,VIX_VIX_Index,Bitcoin_XBTUSD_Curncy
0,2005-01-03,14.08,NaN
1,2005-01-04,13.98,NaN
2,2005-01-05,14.09,NaN
3,2005-01-06,13.58,NaN
4,2005-01-07,13.49,NaN
...,...,...,...
5364,2025-07-25,14.93,117104.31
5365,2025-07-28,15.03,118040.22
5366,2025-07-29,15.98,117479.31
5367,2025-07-30,15.48,117143.62


In [11]:
df_bbg_pmi = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=3,
    rows_to_delete=1,
    sheet_name="PMI", 
    step=1,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=False
)
df_bbg_pmi

,Date,US_PMI_Manufacturing_NAPMPMI_Index_Actual_Economic_Release_Values_ACTUAL_RELEASE,US_PMI_Services_NAPMNMI_Index_Actual_Economic_Release_Values_ACTUAL_RELEASE,Ifo_Index_GRIFPBUS_Index_Actual_Economic_Release_Values_ACTUAL_RELEASE
0,2005-01-04,58.6,NaN,96.2
1,2005-01-05,58.6,NaN,96.2
2,2005-01-06,58.6,NaN,96.2
3,2005-01-07,58.6,NaN,96.2
4,2005-01-10,58.6,NaN,96.2
...,...,...,...,...
5384,2025-08-25,48.0,50.1,NaN
5385,2025-08-26,48.0,50.1,NaN
5386,2025-08-27,48.0,50.1,NaN
5387,2025-08-28,48.0,50.1,NaN


In [12]:
df_bbg_eps = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=2,
    rows_to_delete=0,
    sheet_name="EPS ACWI", 
    step=1,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=False
)
df_bbg_eps

,Date,Earnings_per_Share_Forward_12M_MSCI_ACWI_BEst_EPS
0,2005-07-05,19.6620
1,2005-07-06,19.7054
2,2005-07-07,19.6490
3,2005-07-08,19.6515
4,2005-07-11,19.3912
...,...,...
5237,2025-07-31,49.2064
5238,2025-08-01,49.3141
5239,2025-08-04,49.3998
5240,2025-08-05,49.4122


In [13]:
df_bbg_surprise_index = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=2,
    rows_to_delete=0,
    sheet_name="Economic Surprise Indices", 
    step=1,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=False
)
df_bbg_surprise_index

,Date,Economic_Surprise_EU_CESIEUR_Index,Economic_Surprise_US_CESIUSD_Index,Economic_Surprise_China_CESICNY_Index
0,2005-01-03,-10.7,22.0,11.8
1,2005-01-04,-9.7,22.5,11.5
2,2005-01-05,-8.4,27.6,11.2
3,2005-01-06,-7.6,27.7,10.9
4,2005-01-07,-18.4,26.0,10.6
...,...,...,...,...
5323,2025-07-25,32.3,3.5,3.2
5324,2025-07-28,29.9,6.8,3.0
5325,2025-07-29,29.0,7.8,2.9
5326,2025-07-30,39.7,13.4,2.8


In [14]:
df_bbg_dollar_index = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=1,
    rows_to_delete=0,
    sheet_name="Dollar Index", 
    step=1,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=False
)
df_bbg_dollar_index

,Date,DXY_Curncy
0,2005-01-03,81.300
1,2005-01-04,82.520
2,2005-01-05,82.540
3,2005-01-06,83.150
4,2005-01-07,83.600
...,...,...
5360,2025-07-25,97.645
5361,2025-07-28,98.634
5362,2025-07-29,98.886
5363,2025-07-30,99.815


In [15]:
df_bbg_value_growth = process_multi_header_file(
    file_paths['bbg'], 
    header_rows=2,
    rows_to_delete=0,
    sheet_name="Value_Growth",
    step=1,
    date_col_index=0,
    data_start_col=1,
    interleaved_dates=False
)
df_bbg_value_growth

C:\Users\msark\AppData\Local\Temp\ipykernel_28636\937518599.py:73: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'Date': pd.to_datetime(df_raw.iloc[:, date_col_index], errors='coerce', format=date_format)


,Date,MSCI_ACWI_Value_MXWD000V_Index,MSCI_ACWI_Growth_MXWD000G_Index
0,2005-01-03,148.75,127.64
1,2005-01-04,147.27,126.03
2,2005-01-05,146.38,125.23
3,2005-01-06,146.42,124.99
4,2005-01-07,146,124.92
...,...,...,...
5364,2025-07-25,306.19,655.91
5365,2025-07-28,304.73,656.23
5366,2025-07-29,303.98,652.94
5367,2025-07-30,302.61,653.67


In [16]:
# --- SAVE FINAL DATA ---

dataframes_to_save = {
    'jpmx': df_jpmx,
    'resource_energy': df_resource_energy,
    'resource_metals': df_resource_metals,
    'resource_agrar': df_resource_agrar,
    'country_risk': df_country_risk,
    'bbg_zinsen': df_bbg_zinsen,
    'bbg_spreads': df_bbg_spreads,
    'bbg_inflation': df_bbg_inflation,
    'bbg_btc': df_bbg_btc,
    'bbg_pmi': df_bbg_pmi,
    'bbg_eps': df_bbg_eps,
    'bbg_surprise_index': df_bbg_surprise_index,
    'bbg_dollar_index': df_bbg_dollar_index,
    'bbg_value_growth': df_bbg_value_growth
}

output_hdf_path = os.path.join(data_path, 'cleaned_data.h5')

print(f"Save DataFrames in: {output_hdf_path}")
for key, df in dataframes_to_save.items():
    df.to_hdf(output_hdf_path, key=key, mode='a')
    print(f"- '{key}' saved with shape {df.shape}")

print("\nAll DataFrames successfully saved.")

Save DataFrames in: ..\data\cleaned_data.h5
- 'jpmx' saved with shape (5139, 9)
- 'resource_energy' saved with shape (5316, 7)
- 'resource_metals' saved with shape (6221, 14)
- 'resource_agrar' saved with shape (5190, 8)
- 'country_risk' saved with shape (66, 329)
- 'bbg_zinsen' saved with shape (5369, 6)
- 'bbg_spreads' saved with shape (5341, 3)
- 'bbg_inflation' saved with shape (5364, 3)


C:\Users\msark\AppData\Local\Temp\ipykernel_28636\398380546.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer-float,key->block1_values] [items->Index(['Zinsen_10Y_DE_GDBR10_Index', 'Zinsen_10Y_US_USGG10YR_Index',
       'Zinskurve_2Y10Y_DE_DEYC2Y10_Index',
       'Zinskurve_3M10Y_US_USYC3M10_Index',
       'Zinskurve_2Y10Y_DE_USYC2Y10_Index'],
      dtype='object')]

  df.to_hdf(output_hdf_path, key=key, mode='a')
C:\Users\msark\AppData\Local\Temp\ipykernel_28636\398380546.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer-float,key->block1_values] [items->Index(['VIX_VIX_Index', 'Bitcoin_XBTUSD_Curncy'], dtype='object')]

  df.to_hdf(output_hdf_path, key=key, mode='a')


- 'bbg_btc' saved with shape (5369, 3)
- 'bbg_pmi' saved with shape (5389, 4)
- 'bbg_eps' saved with shape (5242, 2)
- 'bbg_surprise_index' saved with shape (5328, 4)
- 'bbg_dollar_index' saved with shape (5365, 2)
- 'bbg_value_growth' saved with shape (5369, 3)

All DataFrames successfully saved.


C:\Users\msark\AppData\Local\Temp\ipykernel_28636\398380546.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer-float,key->block1_values] [items->Index(['MSCI_ACWI_Value_MXWD000V_Index', 'MSCI_ACWI_Growth_MXWD000G_Index'], dtype='object')]

  df.to_hdf(output_hdf_path, key=key, mode='a')
